In [1]:
!nvidia-smi
!git clone https://github.com/jonathancaevans/BoardBot.git
!sed -i -e 's/yourusername/*yourusername*/g' BoardBot/getClimbs.py
!sed -i -e 's/yourpassword/*yourpassword*/g' BoardBot/getClimbs.py
!python BoardBot/getBoarddims.py
!mkdir placementMaps
!python BoardBot/main.py

Fri Nov 12 20:30:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import pandas as pd
import glob
climbs=[]
for file in glob.glob('/content/placementMaps/*.csv'):
  df=pd.read_csv(file, sep=',',header=None)
  climbs.append(df)

In [2]:
frame = pd.concat(climbs, axis=1, ignore_index=True)
frame = frame.transpose()

In [3]:
frame = frame.drop(0, axis = 1)
print(frame)

         1       2       3       4      5    ...    472    473    474    475    476
0      Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
1      Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
2      Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
3      Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
4      Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
...      ...     ...     ...     ...    ...  ...    ...    ...    ...    ...    ...
56746  Blank   Blank  finish  finish  Blank  ...  Blank  Blank  Blank  Blank  Blank
56747  Blank  finish   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
56748  Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
56749  Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank  Blank
56750  Blank   Blank   Blank   Blank  Blank  ...  Blank  Blank  Blank  Blank

In [4]:
discrete_columns = frame.columns.values.tolist()

In [5]:
!pip install ctgan

In [6]:
def failurerate(ctgan):
  samples = ctgan.sample(100)
  failrate = 0

  for index, row in samples.iterrows():
    fail = 0
    holdTypes = {"Blank":0,"mid":0,"feet":0,"start":0,"finish":0}
    
    for (columnName, columnData) in row.iteritems():
      holdTypes[columnData] = holdTypes.get(columnData,0) + 1
      
    if holdTypes["start"] == 0 or holdTypes["start"] > 2:
      fail = 1
    elif holdTypes["finish"] == 0 or holdTypes["finish"] > 2:
      fail = 1
    elif holdTypes["finish"] + holdTypes["finish"] + holdTypes["finish"] + holdTypes["finish"] > 35:
      fail = 1
    failrate += fail

  return failrate

In [ ]:
from ctgan import CTGANSynthesizer

failurerates = []
ctgan = CTGANSynthesizer(epochs = 60, batch_size = 1000, generator_lr = .0001, discriminator_lr = .0001, save="model.pkl")
ctgan.fit(frame, discrete_columns)
fail = failurerate(ctgan)
print(fail)
failurerates.append(fail)